<a href="https://colab.research.google.com/github/Piaozan/29CM/blob/main/3_%E6%A0%B8%E5%BF%83%E7%94%A8%E6%88%B7%E9%A2%84%E5%A4%84%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import & read file

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime

import warnings
warnings.filterwarnings(action='ignore')

sns.set_palette("RdBu")

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
from google.colab import files
from google.colab import drive
drive.mount("/content/gdrive")

total = pd.read_csv('/content/gdrive/MyDrive/final/data/commerce/commerce2.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# (1) 核心用户定义

- **29CM的会员等级**
- **VIP**
    - 最近6个月购买金额:50万韩元以上
    - 最近6个月1万韩元以上购买次数:10次以上
- **RED**
    - 最近6个月购买金额:30万韩元以上，50万韩元以下
    - 最近6个月1万韩元以上购买次数:5次以上、10次以下
- **ORANGE**
    - 最近6个月购买金额:10万韩元以上，30万韩元以下
    - 最近6个月1万韩元以上购买次数:1次以上，5次以下
- **YELLOW**
    - 最近6个月的购买金额:6个月内没有再购买经验的顾客
- **GREEN**
    - 最近6个月的购买金额:新加入会员专用

- **选择29CM的核心用户**
    - 参考29CM VIP等级,将6个月的标准换算成2个月,定义为**“核心用户”**。

- **核心用户**
    - 2个月购买金额:166000韩元以上
    - 2个月内1万韩元以上购买次数:3次以上

---

# (2) 核心用户 & 非核心用户分类

### 购买者 & 非购买者分类

- 核心用户的最前提条件是购买。
- 因此,首先要了解用户是否在两个月内购买。

In [3]:
# 固有购买者列单
buyer_list = total[total['event_name'] == 'purchase_success'].drop_duplicates(['user_no'], keep='first')

# 购买者为1，否则为Nan
buyer_list['buyer'] = 1

buyer_list = buyer_list[['user_no','buyer']]

In [4]:
# 购买者, 非购买者 区分 : 'buyer' column  购买者 1, 非购买者 0
total = total.merge(buyer_list, on='user_no', how='left')

total['buyer'].fillna(0, inplace=True)

In [5]:
# 不必要的columns 删除
total.drop(['mobile_model_name','mobile_marketing_name','operating_system_version','image_name','category1_code',
            'category2_code','category3_code','brand_no','event_minute','age','birth_date'], axis=1, inplace=True)

In [6]:
total.head(2)

,session_id,event_timestamp,event_name,user_no,item_no,device_type,mobile_brand_name,country,region,platform,item_name,price,category1_name,category2_name,category3_name,brand_name,gender,年龄分类,event_month,event_day,event_weekday,event_hour,week,order_amount,buyer
0,5240d1b731949382c749c79db207b2fa,2021-06-06,click_item,j2PG6ak1chv+aTkyNM3JzA==,JdwoJ4a/Z2YK5nKCKWUiyw==,mobile,Apple,South Korea,Seoul,IOS,WOOL SILK CROP JACKET - SEAL BROWN,135000.0,服装,아우터,재킷,MORE OR LESS,F,20,6,6,6,0,22,135000.0,0.0
1,5ae2c24c1c3e9dc228846c772cfbbbdc,2021-06-06,click_item,tBR8q7ZrRkJwO88bCX6aKA==,V9vAu4jSKMu64NY9zU155Q==,mobile,Apple,South Korea,Seoul,IOS,cl half pants (bone),117000.0,服装,하의,바지,MOIA,F,20,6,6,6,0,22,117000.0,1.0


### 在购买者当中分类核心用户 & 非核心用户

In [7]:
# 2个月的购买人数
print('2个月内29CM购买者人数: ', total[total['buyer'] == 1]['user_no'].nunique())

2个月内29CM购买者人数:  138279


In [8]:
# 购买者购买商品时的行为
user = total[(total['buyer'] == 1) & (total['event_name'] == 'purchase_success')]

# 以购买者为基准购买商品个数(item_no)
user_no = user.groupby(['user_no'])['item_no'].count().reset_index().sort_values('item_no', ascending=False)

# 给所有购买者配1 (以后用于，用户比率与全体用户相比较) (user_cnt)
user_no['user_cnt'] = 1

# 以购买者为基准的购买金额排名 (price)
user_price = user.groupby(['user_no'])['price'].sum().reset_index().sort_values('price', ascending=False)

# 合并数据
user_total = user_no.merge(user_price, on='user_no', how='left')

In [9]:
#  核心用户的条件
#  2个月的支付金额: 166,000韩元以上
#  2个月内1万韩元以上购买次数: 3次以上

# 核心用户 (user_vip)
user_vip = user_total[(( user_total['item_no'] >= 3 ) & ((user_total['price'] / user_total['item_no']) >= 10000))| ( user_total['price'] >= 166000 )]
user_vip = user_vip.sort_values(['price','item_no'], ascending=False)

# 合并于全部用户数据，删除重复用户
user_total = user_vip.append(user_total)
user_total = user_total.drop_duplicates('user_no', keep='first')

# 购买者当中的非核心用户 (user_normal)
user_normal = user_total[59525:]
user_normal = user_normal.sort_values(['price','item_no'], ascending=False)


# user_vip + user_normal 合并 = (在购买者当中分为核心用户以及非核心用户)
user_total = user_vip.append(user_normal)

### 非购买者 => 非核心用户

- 非购买者必须为非核心用户
- 以非购买者的情况填充column
    - 购买数(item_no) = 0
    - 用户(user_cnt) = 1
    - 购买金额(price) = 0
    - 是否核心用户(power_user) = 0

In [10]:
# 非购买者 user_no
unbuyer = total[(total['buyer'] == 0)]

unbuyer = unbuyer.groupby(['user_no'])['item_no'].count().reset_index()
unbuyer['item_no'] = 0
unbuyer['user_cnt'] = 1
unbuyer['price'] = 0

# 购买者 & 非购买者合并
user_total = user_total.append(unbuyer)
user_total

,user_no,item_no,user_cnt,price
0,b2xZVfUtjyGsZckO7j6zkg==,246,1,28841100.0
1,anAN3a+fvEnnvFdQ61+Egw==,203,1,22966700.0
2324,pywOPeCTmJKudBVHC1/5iw==,15,1,11035900.0
6,8j/l4DjsSqXcIR9r5GAmLQ==,115,1,10068900.0
37379,tCbVorRR1nnG7W2rSzxZNQ==,3,1,10008000.0
...,...,...,...,...
105588,zzrlop3HrqxAgIT0VJ9wRg==,0,1,0.0
105589,zzwOfX9q6MNyni9Ff8Ws6A==,0,1,0.0
105590,zzwQnhNQeYnzcsjywnIh3g==,0,1,0.0
105591,zzxlG1wzPCGvsW0SlOleDg==,0,1,0.0


In [11]:
# 全部用户的数量
total['user_no'].nunique()

243872

- 分类核心、非核心用户的所有用户的数量 与 原本数据的全部用户数量 相比，没有差异
- 因此能看出分类上，未出现错误

### 核心用户在全部用户上的占比

- 想要计算核心用户和非核心用户各自的比例，并从中获得洞见。


- **用户数累计比例** => 核心用户在总用户中所占比例
- **用户购买金额累计比例** => 核心用户购买金额在总销售额中所占比例
- **用户购买商品数量累计比例** => 核心用户购买商品数量在总销售量中所占比例

In [12]:
# 用户数累计比例
user_total['user_cnt_cum'] =  user_total['user_cnt'].cumsum()
user_total['user_per_percent'] = user_total['user_cnt'] / len(user_total) * 100
user_total['user_per_percent_cum'] = user_total['user_per_percent'].cumsum()

# 用户购买金额累计比例
user_total['price_cum'] = user_total['price'].cumsum()
user_total['price_per_percent'] = user_total['price'] / sum(user_total['price']) * 100
user_total['price_per_percent_cum'] = user_total['price_per_percent'].cumsum()

# 用户购买商品数量累计比例
user_total['item_no_cum'] = user_total['item_no'].cumsum()
user_total['item_per_percent'] = user_total['item_no'] / sum(user_total['item_no']) * 100
user_total['item_per_percent_cum'] = user_total['item_per_percent'].cumsum()

# 删除不必要的column
user_total = user_total.drop(['user_cnt','item_per_percent','user_cnt_cum','user_per_percent','item_no_cum','price_cum','price_per_percent'], axis=1)

In [13]:
# 排序所有用户
user_total.reset_index(drop=True, inplace=True)

# 核心用户 & 非核心用户
user_vip = user_total[:59525]
user_normal = user_total[59525:]

# 核心用户 & 非核心用户 （包含购买者与非购买者）
user_vip['power_user'] = 1

user_total = user_total.merge(user_vip, on=['user_no', 'item_no', 'price', 'user_per_percent_cum',
       'price_per_percent_cum', 'item_per_percent_cum'], how='left')

user_total['power_user'].fillna(0, inplace=True)

In [14]:
# 查看核心用户在整个行业上的影响
user_total[user_total['power_user'] == 1]

,user_no,item_no,price,user_per_percent_cum,price_per_percent_cum,item_per_percent_cum,power_user
0,b2xZVfUtjyGsZckO7j6zkg==,246,28841100.0,0.000410,0.113023,0.057958,1.0
1,anAN3a+fvEnnvFdQ61+Egw==,203,22966700.0,0.000820,0.203025,0.105785,1.0
2,pywOPeCTmJKudBVHC1/5iw==,15,11035900.0,0.001230,0.246273,0.109319,1.0
3,8j/l4DjsSqXcIR9r5GAmLQ==,115,10068900.0,0.001640,0.285731,0.136414,1.0
4,tCbVorRR1nnG7W2rSzxZNQ==,3,10008000.0,0.002050,0.324950,0.137121,1.0
...,...,...,...,...,...,...,...
59520,DyasvMcma5FbF2AHHu+ubw==,3,30000.0,24.406656,80.056593,74.633167,1.0
59521,2y/T4u1hI1U5+ww0mqLdnA==,3,30000.0,24.407066,80.056711,74.633874,1.0
59522,I56mmeC+DT7GueHwbOVeow==,3,30000.0,24.407476,80.056828,74.634581,1.0
59523,7/PpFKdDqzuxcluFxAryyQ==,3,30000.0,24.407886,80.056946,74.635288,1.0


- user_per_percent_cum : 全部用户中核心用户的比例
    - 全部用户中约 **24%是核心用户**。

- price_per_percent_cum : 核心用户在总销售额中的占比
    - 总销售额中约 **80%由核心用户贡献**。

- item_per_percent_cum : 核心用户在总销售量中购买的商品数量占比
    - 总销售量中约 **75%的商品被核心用户购买**。

- **即，约24%的用户负责了全部80%的销售额。**
- **“核心用户”中能发现“二八原则”。**
- **因此，29CM应该了解这些核心用户的特征，并持续地以他们为目标，保持他们的忠诚度，这对盈利有很大的帮助。**

# 核心用户 & 非核心用户 （数据处理）

## 合并数据

- 为了更详细的比较，在整个数据total中查看
- 为了区分核心、非核心用户，对此进行数据处理

In [15]:
# 删除不必要的columns
user_cum = user_total.drop(['user_per_percent_cum','price_per_percent_cum','item_per_percent_cum'], axis=1)
user_cum.columns = ['user_no','item_per_cnt','total_per_price','power_user']

In [16]:
# total & user_cum 合并, 命名为final
final = total.merge(user_cum, on='user_no', how='left')

In [17]:
# 检查是否合并成功，查看缺失值
final['user_no'].isnull().sum()

0

In [18]:
final.head(2)

,session_id,event_timestamp,event_name,user_no,item_no,device_type,mobile_brand_name,country,region,platform,item_name,price,category1_name,category2_name,category3_name,brand_name,gender,年龄分类,event_month,event_day,event_weekday,event_hour,week,order_amount,buyer,item_per_cnt,total_per_price,power_user
0,5240d1b731949382c749c79db207b2fa,2021-06-06,click_item,j2PG6ak1chv+aTkyNM3JzA==,JdwoJ4a/Z2YK5nKCKWUiyw==,mobile,Apple,South Korea,Seoul,IOS,WOOL SILK CROP JACKET - SEAL BROWN,135000.0,服装,아우터,재킷,MORE OR LESS,F,20,6,6,6,0,22,135000.0,0.0,0,0.0,0.0
1,5ae2c24c1c3e9dc228846c772cfbbbdc,2021-06-06,click_item,tBR8q7ZrRkJwO88bCX6aKA==,V9vAu4jSKMu64NY9zU155Q==,mobile,Apple,South Korea,Seoul,IOS,cl half pants (bone),117000.0,服装,하의,바지,MOIA,F,20,6,6,6,0,22,117000.0,1.0,1,128000.0,0.0


## 计算每个用户的各种行为次数

- 每个用户都被分为核心用户和非核心用户，因此他们的行为模式之间可能存在很大差异。
- 因此，我们想通过分析每个用户的行为、性别、年龄、商品分类、不同星期的购买次数等数据，了解它们之间的差异。

### event（行为）

In [19]:
# 各用户的总点击数
click_cnt = final[final['event_name'] == 'click_item'].groupby(['user_no'])['item_no'].count().reset_index().rename(columns = {'item_no' : 'click_cnt'})

# 各用户的总购物车数
cart_cnt = final[final['event_name'] == 'add_to_cart'].groupby(['user_no'])['item_no'].count().reset_index().rename(columns = {'item_no' : 'cart_cnt'})

# 各用户的总收藏数
like_cnt = final[final['event_name'] == 'like_item'].groupby(['user_no'])['item_no'].count().reset_index().rename(columns = {'item_no' : 'like_cnt'})

In [20]:
click_cnt

,user_no,click_cnt
0,+++T9c17sr6wEYOy9UFv3w==,2
1,+++aQH5NnhiU/9qECKk3uQ==,49
2,++0bwiYVJ/TAsCpxI4UCuw==,50
3,++292nTvV+bCI1Diuw96GA==,1
4,++2b+ESK8hAPty/PmYpJCA==,3
...,...,...
193130,zzwQnhNQeYnzcsjywnIh3g==,3
193131,zzwYai1YDr0JZQwaTqc0mQ==,7
193132,zzwxh/ZL9TPJOWZHLMvnDQ==,11
193133,zzxlG1wzPCGvsW0SlOleDg==,1


In [21]:
# 合并数据 (with user_total)
user_total = user_total.merge(click_cnt, on='user_no', how='left')
user_total = user_total.merge(cart_cnt, on='user_no', how='left')
user_total = user_total.merge(like_cnt, on='user_no', how='left')

In [22]:
user_total

,user_no,item_no,price,user_per_percent_cum,price_per_percent_cum,item_per_percent_cum,power_user,click_cnt,cart_cnt,like_cnt
0,b2xZVfUtjyGsZckO7j6zkg==,246,28841100.0,0.00041,0.113023,0.057958,1.0,7534.0,NaN,NaN
1,anAN3a+fvEnnvFdQ61+Egw==,203,22966700.0,0.00082,0.203025,0.105785,1.0,325.0,105.0,NaN
2,pywOPeCTmJKudBVHC1/5iw==,15,11035900.0,0.00123,0.246273,0.109319,1.0,55.0,NaN,NaN
3,8j/l4DjsSqXcIR9r5GAmLQ==,115,10068900.0,0.00164,0.285731,0.136414,1.0,119.0,265.0,NaN
4,tCbVorRR1nnG7W2rSzxZNQ==,3,10008000.0,0.00205,0.324950,0.137121,1.0,3.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...
243867,zzrlop3HrqxAgIT0VJ9wRg==,0,0.0,99.99836,100.000000,100.000000,0.0,1.0,NaN,NaN
243868,zzwOfX9q6MNyni9Ff8Ws6A==,0,0.0,99.99877,100.000000,100.000000,0.0,16.0,NaN,NaN
243869,zzwQnhNQeYnzcsjywnIh3g==,0,0.0,99.99918,100.000000,100.000000,0.0,3.0,NaN,NaN
243870,zzxlG1wzPCGvsW0SlOleDg==,0,0.0,99.99959,100.000000,100.000000,0.0,1.0,NaN,NaN


In [23]:
# 填充行为的缺失值（点击、购物车、收藏）
user_total.fillna(0, inplace=True)
user_total.isnull().sum().sum()

0

### 基础信息

In [24]:
# 创建有关个用户的基础信息table
user_info = final[['user_no','buyer','mobile_brand_name','region','platform','gender','年龄分类']]
user_info.drop_duplicates('user_no', keep='first', inplace=True)

In [25]:
# 与user_total合并
user_total = user_total.merge(user_info, on='user_no', how='left')
user_total.head(2)

,user_no,item_no,price,user_per_percent_cum,price_per_percent_cum,item_per_percent_cum,power_user,click_cnt,cart_cnt,like_cnt,buyer,mobile_brand_name,region,platform,gender,年龄分类
0,b2xZVfUtjyGsZckO7j6zkg==,246,28841100.0,0.00041,0.113023,0.057958,1.0,7534.0,0.0,0.0,1.0,Apple,Chungcheongnam-do,IOS,F,40
1,anAN3a+fvEnnvFdQ61+Egw==,203,22966700.0,0.00082,0.203025,0.105785,1.0,325.0,105.0,0.0,1.0,Samsung,Seoul,ANDROID,F,40


### 商品分类（购买次数）

In [26]:
final['category1_name'].value_counts()

服装        3255145
杂货        1349863
生活/健康      234897
家居         190674
数码/家电      123827
化妆品/美容      66095
食品          42477
运动/休闲       39716
文化          29803
儿童          10097
家畜           8447
成人节          3184
Name: category1_name, dtype: int64

In [27]:
# 所有用户ID
all_user_no = final[['user_no']]

# 购买服装的table
clothes_cnt = final[(final['category1_name'] == '服装') & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : '服装'})

# 所有用户ID & 购买服装 合并
category_p_cnt = all_user_no.merge(clothes_cnt,on='user_no', how='left')

In [28]:
# 各用户按商品分类进行的购买个数table
for category in final['category1_name'].unique()[1:]:
    category_cnt =  final[(final['category1_name'] == category) & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : category})
    category_p_cnt = category_p_cnt.merge(category_cnt, on='user_no', how='left')

# 非购买者，用0来填充缺失值
category_p_cnt.fillna(0, inplace=True)
# 删除重复用户ID
category_p_cnt.drop_duplicates('user_no', keep='first', inplace=True)

category_p_cnt.drop(np.nan, axis=1, inplace=True)

In [29]:
# (2个月间) 各用户按商品分类的购买数量
category_p_cnt.head(3)

,user_no,服装,生活/健康,运动/休闲,杂货,化妆品/美容,家居,文化,数码/家电,食品,儿童,家畜,成人节
0,j2PG6ak1chv+aTkyNM3JzA==,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tBR8q7ZrRkJwO88bCX6aKA==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,qtcoyasZW4XF7NpbcSB4mg==,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# 商品分类购买数量 & user_total合并
user_total = user_total.merge(category_p_cnt, on='user_no', how='left')

### 星期（购买数量）

In [31]:
# 所有用户ID
all_user_no = final[['user_no']]

# 周日的时候，购买的用户table
sunday_cnt = final[(final['event_weekday'] == 6) & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : '일요일'})

# 合并
week_p_cnt = all_user_no.merge(sunday_cnt,on='user_no', how='left')

In [33]:
# 各用户按照星期购买的商品数量
for week in final['event_weekday'].unique()[1:]:
    week_cnt =  final[(final['event_weekday'] == week) & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : week})
    week_p_cnt = week_p_cnt.merge(week_cnt, on='user_no', how='left')

# 非购买者用0来填充缺失值
week_p_cnt.fillna(0, inplace=True)
# 删除重复用户ID
week_p_cnt.drop_duplicates('user_no', keep='first', inplace=True)

# 修改星期的名字
week_p_cnt.columns = ['user_no','Sun','Mon','Tue','Wed','Thu','Fri', 'Sat']

In [36]:
# 星期购买的商品数量与user_total合并

user_total = user_total.merge(week_p_cnt, on='user_no', how='left')

In [37]:
user_total.head(2)

,user_no,item_no,price,user_per_percent_cum,price_per_percent_cum,item_per_percent_cum,power_user,click_cnt,cart_cnt,like_cnt,buyer,mobile_brand_name,region,platform,gender,年龄分类,服装,生活/健康,运动/休闲,杂货,化妆品/美容,家居,文化,数码/家电,食品,儿童,家畜,成人节,Sun,Mon,Tue,Wed,Thu,Fri,Sat
0,b2xZVfUtjyGsZckO7j6zkg==,246,28841100.0,0.00041,0.113023,0.057958,1.0,7534.0,0.0,0.0,1.0,Apple,Chungcheongnam-do,IOS,F,40,238.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,23.0,42.0,31.0,37.0,34.0,50.0
1,anAN3a+fvEnnvFdQ61+Egw==,203,22966700.0,0.00082,0.203025,0.105785,1.0,325.0,105.0,0.0,1.0,Samsung,Seoul,ANDROID,F,40,171.0,9.0,2.0,20.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,0.0,0.0,13.0


### 各用户的客单价

In [38]:
# 客单价 item_per_price 计算
user_total['item_per_price'] = user_total['price'] / user_total['item_no'] 
user_total['item_per_price'].fillna(0, inplace=True)

# 购买商品数量修改名称, item_no => buy_cnt
user_total.rename(columns = {'item_no' : 'buy_cnt'}, inplace=True)

### 时间

In [39]:
# 所有用户ID
all_user_no = final[['user_no']]

# 0点 购买者 table
zero_cnt = final[(final['event_hour'] == 0) & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : '0시'})

# 合并
hour_p_cnt = all_user_no.merge(zero_cnt,on='user_no', how='left')

In [40]:
# 各用户在各个时间段购买商品数
for hour in final['event_hour'].unique()[1:]:
    hour_cnt =  final[(final['event_hour'] == hour) & (final['event_name'] == 'purchase_success')].groupby(['user_no'])['item_no'].count().reset_index().rename(columns={'item_no' : hour})
    hour_p_cnt = hour_p_cnt.merge(hour_cnt, on='user_no', how='left')

#非购买者把缺失值填充为0
hour_p_cnt.fillna(0, inplace=True)
# 删除重复用户ID
hour_p_cnt.drop_duplicates('user_no', keep='first', inplace=True)
# 修改时间段名称
hour_p_cnt.columns = ['user_no','0H','1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','13H','14H','15H','16H','17H','18H','19H','20H','21H','22H','23H']

In [41]:
# user_total与时间段购买商品数合并
user_total = user_total.merge(hour_p_cnt, on='user_no', how='left')

In [44]:
# 删除不必要的column
user_total.drop(['user_per_percent_cum','price_per_percent_cum','item_per_percent_cum'], axis=1, inplace=True)

In [46]:
user_total.head(3)

,user_no,buy_cnt,price,power_user,click_cnt,cart_cnt,like_cnt,buyer,mobile_brand_name,region,platform,gender,年龄分类,服装,生活/健康,运动/休闲,杂货,化妆品/美容,家居,文化,数码/家电,食品,儿童,家畜,成人节,Sun,Mon,Tue,Wed,Thu,Fri,Sat,item_per_price,0H,1H,2H,3H,4H,5H,6H,7H,8H,9H,10H,11H,12H,13H,14H,15H,16H,17H,18H,19H,20H,21H,22H,23H
0,b2xZVfUtjyGsZckO7j6zkg==,246,28841100.0,1.0,7534.0,0.0,0.0,1.0,Apple,Chungcheongnam-do,IOS,F,40,238.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,23.0,42.0,31.0,37.0,34.0,50.0,117240.243902,7.0,8.0,5.0,0.0,0.0,0.0,10.0,6.0,12.0,13.0,12.0,13.0,10.0,14.0,12.0,15.0,19.0,19.0,18.0,8.0,17.0,6.0,15.0,7.0
1,anAN3a+fvEnnvFdQ61+Egw==,203,22966700.0,1.0,325.0,105.0,0.0,1.0,Samsung,Seoul,ANDROID,F,40,171.0,9.0,2.0,20.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,0.0,0.0,13.0,113136.453202,13.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.0
2,pywOPeCTmJKudBVHC1/5iw==,15,11035900.0,1.0,55.0,0.0,0.0,1.0,Apple,Seoul,IOS,M,30,7.0,4.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,8.0,4.0,0.0,0.0,735726.666667,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0


---

In [45]:
user_total.to_csv('/content/gdrive/MyDrive/final/data/commerce/user_total.csv', index=False)